In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import time
import math
import seaborn as sns
import pandas as pd
import numpy as np
import scipy as sci
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from matplotlib import pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#import data
test = pd.read_csv('../input/test.csv')
train = pd.read_csv('../input/train.csv')
combine = [train,test]

In [ ]:
#func for observing data from https://www.kaggle.com/nholloway/catboost-v-xgboost-v-lightgbm
def meet_ur_data(df):
    df = df
    description = pd.DataFrame(index=['observations(rows)', 'percent missing', 'dtype', 'range'])
    numerical = []
    categorical = []
    for col in df.columns:
        obs = df[col].size
        p_nan = round(df[col].isna().sum()/obs, 2)
        num_nan = f'{p_nan}% ({df[col].isna().sum()}/{obs})'
        dtype = 'categorical' if df[col].dtype == object else 'numerical'
        numerical.append(col) if dtype == 'numerical' else categorical.append(col)
        rng = f'{len(df[col].unique())} labels' if dtype == 'categorical' else f'{df[col].min()}-{df[col].max()}'
        description[col] = [obs, num_nan, dtype, rng]

    pd.set_option('display.max_columns', 100)
    display(description)
    display(df.head())

In [ ]:
meet_ur_data(train)

Sex, and Embarked transform to numerical sex=(male = 1, female = 0), Embarked = (S=0, C=1, Q=2)

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map({'female': 0,'male': 1}).astype(int)
freq_port = train.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train.head()

In [ ]:
meet_ur_data(train)

In [ ]:
features = ['Survived', 'Pclass', 'Sex', 'SibSp', 'Fare', 'Embarked','Parch']
pairplot = sns.pairplot(train[features], diag_kind='kde', hue='Survived')
plt.show()

In [ ]:
trace1 = go.Heatmap(
    z = np.absolute(train[features].astype(float).corr().values),
    x = train[features].columns.values,
    y = train[features].columns.values,
    colorscale = 'Portland', 
    reversescale = False, 
    opacity = 1.0)
        
data = [trace1]
layout = go.Layout(
    title = 'Correlation map (Numerical Features)',
    xaxis = dict(ticks = '', nticks = 36),
    yaxis = dict(ticks = ''),
    width = 700, height = 700
)

fig = go.Figure(data = data, layout = layout)
py.iplot(fig)

In [ ]:
train = train.drop(['Ticket','Cabin','PassengerId','Name'],axis = 1)
test = test.drop(['Ticket','Cabin','Name'],axis = 1)
combine = [test,train]
train.head()

In [ ]:
meet_ur_data(train)

In [ ]:
#filling Age
guess_ages = np.zeros((2,3))
guess_ages
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

In [ ]:
meet_ur_data(train)

In [ ]:
features_2 = ['Survived', 'Pclass', 'Sex', 'SibSp', 'Fare', 'Embarked','Parch', 'Age']
trace1 = go.Heatmap(
    z = np.absolute(train[features_2].astype(float).corr().values),
    x = train[features_2].columns.values,
    y = train[features_2].columns.values,
    colorscale = 'Portland', 
    reversescale = False, 
    opacity = 1.0)
        
data = [trace1]
layout = go.Layout(
    title = 'Correlation map (Numerical Features)',
    xaxis = dict(ticks = '', nticks = 36),
    yaxis = dict(ticks = ''),
    width = 700, height = 700
)

fig = go.Figure(data = data, layout = layout)
py.iplot(fig)

In [ ]:
X_train = train.drop("Survived", axis=1)
Y_train = train["Survived"]
X_test  = test.drop("PassengerId", axis=1).copy()

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from hyperopt import hp, tpe, Trials, STATUS_OK
from hyperopt import fmin

y_df = Y_train.copy()
x_df = X_train.copy()
train_x, test_x, train_y, test_y = train_test_split(x_df, y_df, test_size=0.20)

In [52]:
def org_results(trials, hyperparams, model_name):
    fit_idx = -1
    for idx, fit  in enumerate(trials):
        hyp = fit['misc']['vals']
        xgb_hyp = {key:[val] for key, val in hyperparams.items()}
        if hyp == xgb_hyp:
            fit_idx = idx
            break
            
    train_time = str(trials[-1]['refresh_time'] - trials[0]['book_time'])
    acc = round(trials[fit_idx]['result']['accuracy'], 3)
    train_auc = round(trials[fit_idx]['result']['train auc'], 3)
    test_auc = round(trials[fit_idx]['result']['test auc'], 3)

    results = {
        'model': model_name,
        'parameter search time': train_time,
        'accuracy': acc,
        'test auc score': test_auc,
        'training auc score': train_auc,
        'parameters': hyperparams
    }
    return results

In [54]:
def xgb_objective(space, early_stopping_rounds=50):
    
    model = XGBClassifier(
        learning_rate = space['learning_rate'], 
        n_estimators = int(space['n_estimators']), 
        max_depth = int(space['max_depth']), 
        min_child_weight = space['m_child_weight'], 
        gamma = space['gamma'], 
        subsample = space['subsample'], 
        colsample_bytree = space['colsample_bytree'],
        objective = 'binary:logistic'
    )

    model.fit(train_x, train_y, 
              eval_set = [(train_x, train_y), (test_x, test_y)],
              eval_metric = 'auc',
              early_stopping_rounds = early_stopping_rounds,
              verbose = False)
     
    predictions = model.predict(test_x)
    test_preds = model.predict_proba(test_x)[:,1]
    train_preds = model.predict_proba(train_x)[:,1]
    
    xgb_booster = model.get_booster()
    train_auc = roc_auc_score(train_y, train_preds)
    test_auc = roc_auc_score(test_y, test_preds)
    accuracy = accuracy_score(test_y, predictions) 

    return {'status': STATUS_OK, 'loss': 1-test_auc, 'accuracy': accuracy,
            'test auc': test_auc, 'train auc': train_auc
           }
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 1000, 25),
    'max_depth': hp.quniform('max_depth', 1, 12, 1),
    'm_child_weight': hp.quniform('m_child_weight', 1, 6, 1),
    'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'learning_rate': hp.loguniform('learning_rate', np.log(.001), np.log(.3)),
    'colsample_bytree': hp.quniform('colsample_bytree', .5, 1, .1)
}
trials = Trials()
xgb_hyperparams = fmin(fn = xgb_objective, 
                 max_evals = 1000, 
                 trials = trials,
                 algo = tpe.suggest,
                 space = space
                )

100%|██████████| 1000/1000 [05:54<00:00,  2.04it/s, best loss: 0.06913931248384597]


In [53]:
xgb_results = org_results(trials.trials, xgb_hyperparams, 'XGBoost')
display(xgb_results)

{'model': 'XGBoost',
 'parameter search time': '0:00:20.879000',
 'accuracy': 0.86,
 'test auc score': 0.918,
 'training auc score': 0.98,
 'parameters': {'colsample_bytree': 0.9,
  'gamma': 0.55,
  'learning_rate': 0.1885950802549632,
  'm_child_weight': 4,
  'max_depth': 11,
  'n_estimators': 300,
  'subsample': 0.75}}

In [ ]:
params = xgb_hyperparams
params.update({ 'm_child_weight': 4,
              'n_estimators': 300,
               'max_depth': 11,
              })

NEO = XGBClassifier(**params)

NEO.fit(X_train, Y_train, 
        verbose = False)
    
Y_pred = NEO.predict(X_test)

submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": Y_pred
    })

In [ ]:
submission.head()

In [ ]:
#save
submission.to_csv('submission.csv',index=False)